# Creación de conversaciones ARP

### Descripción: 
Con este código se va a intentar crear todos los pasos de una conversación ARP, mediante IA generativa. Se pretende pasar una descripción de esta conversación al modelo y que este sea capaz de crear todos los pasos/paquetes necesarios para replicar una conversación con esas mismas especificaciones

In [ ]:
import random
import re
import pickle
import time
import os 

import openai
from openai import OpenAI
os.environ['OPENAI_API_KEY'] = 'FILL'
client = OpenAI()

from scapy.all import *
from scapy.utils import RawPcapReader, wrpcap
import scapy.all as scapy

from scapy.layers.inet import IP, ICMP

openai.__version__

C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated
  cipher=algorithms.Blowfish,
C:\Users\adayd\anaconda3\envs\PAC-GPT\Lib\site-packages\scapy\layers\ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated
  cipher=algorithms.CAST5,


'1.10.0'

## Generación de la descipción de una comunicación ARP

<center><img src="https://lh3.googleusercontent.com/proxy/5BFPjEuLzirf_rQUpK9oPibFDhfXjP7eez9zJdPXfEl5QZyEp6jLTDSpVc8RBygmwlQ5s4hV-V_muXLdOGmmrsjD-JaqO4HtlQHkO1D8Tj8">
    
<center> Esta imagen es genérica, nosotros crearemos conversaciones que sean tanto una pregunta a broadcast como a una ubicación específica.
<br>
    ---
<br>Una conversación normal ARP tiene una petición (opcode = 1) en la que una fuente pregunta a broadcast o a otro elemento de la red por una IP en concreto, y diciendo la IP a la que comunicar la respuesta.
La respuesta tiene (opcode = 2), y tiene como fuente las respuestas, mientras que como destino tiene al que inicia la conversación.   
<center>---

La primera query (query1) representa una conexión ARP sencilla donde se envía la petición a un equipo específico, la segunda (query2) es más complicada, enviando la solicitud a broadcast.

In [2]:
system_message = "This is an ICMP Echo conversation summary:\n"
system_message += 'Source: IP="192.168.1.10" // Destination: IP="192.168.1.20"// Others: id=0x0045 seq= 81 ttl=56 type=Echo"\n'
system_message += "This is the python code to generate the request and reply packets of the conversation, respectively:\n"
system_message += 'pkt_1=scapy.IP(src="192.168.1.10", dst="192.168.1.20", ttl=56)/scapy.ICMP(type=8, id=0x0045, seq=81)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))'
system_message += 'pkt_2=scapy.IP(src="192.168.1.20", dst="192.168.1.10", ttl=56)/scapy.ICMP(type=0, id=0x0045, seq=81)\n'

system_message = "This is an ICMP Timestamp conversation summary:\n"
system_message += 'Source: IP="222.240.4.204" // Destination: IP="192.168.1.20"// Others: id=0xFF62 seq= 4311 ttl=56 type=Timestamp"\n'
system_message += "This is the python code to generate the request and reply packets of the conversation, respectively:\n"
system_message += 'pkt_1=scapy.IP(src="222.240.4.204", dst="200.40.184.21", ttl=56)/scapy.ICMP(type=13, id=0xFF62, seq=4311)\n'
system_message += 'time.sleep(abs(random.gauss(0, 0.03)))'
system_message += 'pkt_2=scapy.IP(src="200.40.184.21", dst="222.240.4.204", ttl=56)/scapy.ICMP(type=14, id=0xFF62, seq=4311)\n'

## Carga de posibles conversaciones

In [3]:
with open("../../data/new/ICMP_Conv_Testing_v2.txt","r",encoding="utf8") as f:
    Conv_summaries = f.read().splitlines()
    
print("Número de resúmenes: " +str(len(Conv_summaries)))

Número de resúmenes: 280


### Carga de posibles respuestas anteriores para encadenarlas

In [4]:
class response():
    "Stores name and place pairs"
    def __init__(self, name, place):
        self.prompt_summary = name
        self.completion = place

In [5]:
try:
    responses = pickle.load( open("../../data/Conversations/ICMP/pickle/ICMP_pairs_Aday_v3.pkl", "rb" ))
except:
    responses = []

print("Number of previous responses: " + str(len(responses)))

Number of previous responses: 0


## Generación de comandos de paquetes de conversación

In [6]:
i=0
for i in range(200):
   
    summary = Conv_summaries[i]   
    print("...........................................")
    print( " GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter " +str(i+1) +")")
    

    ## Luego seleccionas aleatoriamente resumenes ya generados para pedirle que te saque el comando a partir del paquete
    #summaries = random.choices(Conv_summary,k=summaries_per_request)
    text_sum = ""
    
    query_content = "This is a conversation summary:\n"
    query_content += summary + "\n"
    text_sum += summary + "\n"
    query_content += "\nGenerate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.\n"
    query_content += "Use the following structure:\n"
    query_content += "pkt_1=...\n"
    query_content += "time.sleep(abs(random.gauss(0, 0.03)))\n"
    query_content += "pkt_2=...\n"
    
    #Query es el mensaje que le envias al modelo
    print(query_content)

    messages=[{"role": "system", "content": system_message},
        {"role": "user", "content": query_content}]
    
    #Esta es la petición que le mandas al modelo de IA
    completion = client.chat.completions.create(
        model="gpt-4o-mini", 
        messages=messages,
        max_tokens=2600,
        temperature=0.1)
    
    print("...Generated!")
    
    # Guardamos tanto el resumen de los paquetes que solicitamos y lo que devuelve el modelo para estos paquetes
    responses.append(response(text_sum, completion))
    
    codigo = responses[-1].completion.choices[0].message.content
    
    if codigo.startswith("```python"):
        codigo = codigo[len("```python"):].lstrip()  # Elimina ```python al inicio
    if codigo.endswith("```"):
        codigo = codigo[:-len("```")].rstrip()  # Elimina ``` al final
    
    exec(codigo)
    
    with open("../../data/Conversations/ICMP/pcap/ICMP_Conv_generated_gpt_4omini_v3.pcap", "ba+") as f:
        wrpcap(f, pkt_1, append=True)
    
    with open("../../data/Conversations/ICMP/pcap/ICMP_Conv_generated_gpt_4omini_v3.pcap", "ba+") as f:
        wrpcap(f, pkt_2, append=True)
        
    # Guardamos tanto la petición como la respuesta en un archivo
    pickle.dump(responses, open( "../../data/Conversations/ICMP/pickle/ICMP_pairs_Aday_v3.pkl", "wb" ) )
    
    print("Number of completitions done: " + str(len(responses)))

...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 1)
This is a conversation summary:
Source: IP="43.241.136.122" // Destination: IP="199.105.28.73" // Others: id=0x5D4A seq= 3120 type=Echo

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 1
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 2)
This is a conversation summary:
Source: IP="194.237.112.92" // Destination: IP="122.89.246.255" // Others: id=0x8E3F seq= 1784 type=Timestamp

Generate python code for creating the packets in the conversation with scapy framework.Don´t 

...Generated!
Number of completitions done: 13
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 14)
This is a conversation summary:
Source: IP="229.172.161.182" // Destination: IP="197.51.252.48" // Others: id=0x6B3E seq= 1871 type=Timestamp

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 14
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 15)
This is a conversation summary:
Source: IP="54.151.120.44" // Destination: IP="108.60.243.63" // Others: id=0x8D21 seq= 2728 type=Echo

Generate python code for creating the packet

...Generated!
Number of completitions done: 26
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 27)
This is a conversation summary:
Source: IP="11.6.14.34" // Destination: IP="22.26.34.40" // Others: id=0x1002 seq= 1074 type=Echo

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 27
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 28)
This is a conversation summary:
Source: IP="11.9.21.45" // Destination: IP="22.39.51.59" // Others: id=0x1003 seq= 1111 type=Timestamp

Generate python code for creating the packets in the con

...Generated!
Number of completitions done: 39
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 40)
This is a conversation summary:
Source: IP="11.45.105.177" // Destination: IP="22.195.255.27" // Others: id=0x100F seq= 1555 type=Timestamp

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 40
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 41)
This is a conversation summary:
Source: IP="11.48.112.188" // Destination: IP="22.208.16.46" // Others: id=0x1010 seq= 1592 type=Echo

Generate python code for creating the packets i

...Generated!
Number of completitions done: 52
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 53)
This is a conversation summary:
Source: IP="11.84.196.68" // Destination: IP="22.108.220.32" // Others: id=0x101C seq= 2036 type=Echo

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 53
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 54)
This is a conversation summary:
Source: IP="11.87.203.79" // Destination: IP="22.121.237.51" // Others: id=0x101D seq= 2073 type=Timestamp

Generate python code for creating the packets in

...

...Generated!
Number of completitions done: 143
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 144)
This is a conversation summary:
Source: IP="11.101.69.57" // Destination: IP="22.11.242.29" // Others: id=0x1077 seq= 5403 type=Timestamp

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 144
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 145)
This is a conversation summary:
Source: IP="11.104.76.68" // Destination: IP="22.24.3.48" // Others: id=0x1078 seq= 5440 type=Echo

Generate python code for creating the packets in

...Generated!
Number of completitions done: 156
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 157)
This is a conversation summary:
Source: IP="11.140.160.200" // Destination: IP="22.180.208.26" // Others: id=0x1084 seq= 5884 type=Echo

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 157
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 158)
This is a conversation summary:
Source: IP="11.143.167.211" // Destination: IP="22.193.225.45" // Others: id=0x1085 seq= 5921 type=Timestamp

Generate python code for creating the pa

...Generated!
Number of completitions done: 169
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 170)
This is a conversation summary:
Source: IP="11.179.251.91" // Destination: IP="22.93.173.27" // Others: id=0x1091 seq= 6365 type=Timestamp

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 170
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 171)
This is a conversation summary:
Source: IP="11.182.2.102" // Destination: IP="22.106.190.46" // Others: id=0x1092 seq= 6402 type=Echo

Generate python code for creating the packet

...Generated!
Number of completitions done: 182
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 183)
This is a conversation summary:
Source: IP="11.218.86.234" // Destination: IP="22.6.138.30" // Others: id=0x109E seq= 6846 type=Echo

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 183
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 184)
This is a conversation summary:
Source: IP="11.221.93.245" // Destination: IP="22.19.155.49" // Others: id=0x109F seq= 6883 type=Timestamp

Generate python code for creating the packets

...Generated!
Number of completitions done: 195
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 196)
This is a conversation summary:
Source: IP="11.1.177.123" // Destination: IP="22.175.103.25" // Others: id=0x10AB seq= 7327 type=Timestamp

Generate python code for creating the packets in the conversation with scapy framework.Don´t give back any comments, just bare code.Dont give back any comments or imports, just bare code and commands. Dont start the resposnses wiht ```python This is a flagrant error.
Use the following structure:
pkt_1=...
time.sleep(abs(random.gauss(0, 0.03)))
pkt_2=...

...Generated!
Number of completitions done: 196
...........................................
 GENERATING SCAPY COMMANDS FOR THE FOLLOWING CONVERSATIONS (iter 197)
This is a conversation summary:
Source: IP="11.4.184.134" // Destination: IP="22.188.120.44" // Others: id=0x10AC seq= 7364 type=Echo

Generate python code for creating the packet

In [72]:
# Este es el resumen de lo que está en el struct de responses para la última petición
print(responses[-1].prompt_summary)
print(responses[-1].completion.choices[0].text)

Source: IP="192.168.1.1" // Destination: IP="192.168.1.2" // Others: id=0x4952 seq= 1303



pkt_list = []

# Request packet
request_pkt = scapy.IP(src="192.168.1.1", dst="192.168.1.2")/scapy.ICMP(type=8, id=0x4952, seq=1303)
pkt_list.append(request_pkt)

# Reply packet
reply_pkt = scapy.IP(src="192.168.1.2", dst="192.168.1.1")/scapy.ICMP(type=0, id=0x4952, seq=1303)
pkt_list.append(reply_pkt)


In [75]:
#Esta es la forma que tiene una respuesta del modelo de IA
print(responses[-1].completion)

Completion(id='cmpl-9GtCQbzvShHTmYNT96kDftxV8YxqR', choices=[CompletionChoice(finish_reason='stop', index=0, logprobs=None, text='\n\npkt_list = []\n\n# Request packet\nrequest_pkt = scapy.IP(src="192.168.1.1", dst="192.168.1.2")/scapy.ICMP(type=8, id=0x4952, seq=1303)\npkt_list.append(request_pkt)\n\n# Reply packet\nreply_pkt = scapy.IP(src="192.168.1.2", dst="192.168.1.1")/scapy.ICMP(type=0, id=0x4952, seq=1303)\npkt_list.append(reply_pkt)')], created=1713812462, model='gpt-3.5-turbo-instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=119, prompt_tokens=230, total_tokens=349))


### Generación de paquetes

In [78]:
# Ejecutamos los comandos que devuelve el modelo para generar los paquetes
exec(responses[-1].completion.choices[0].text)
print(pkt_list)

[<IP  frag=0 proto=icmp src=192.168.1.1 dst=192.168.1.2 |<ICMP  type=echo-request id=0x4952 seq=0x517 |>>, <IP  frag=0 proto=icmp src=192.168.1.2 dst=192.168.1.1 |<ICMP  type=echo-reply id=0x4952 seq=0x517 |>>]


In [80]:
#Enviarlos para probar si se envian correctamente
send(pkt_list)


Sent 2 packets.


### Introducción en un pcap

In [83]:
# Guardamos los paquetes generados en un pcap
with open("../../data/Conversations/ICMP/pcap/ICMPgenerated_3_5_turbo-instruct_conv.pcap", "wb") as f:
    wrpcap(f, pkt_list)